In [63]:
from torch_geometric.data import ClusterData, ClusterLoader
import torch
from ogb.linkproppred import PygLinkPropPredDataset
from torch_geometric.utils import degree

In [64]:
from torch import Tensor
def reverseperm(perm: Tensor):
    N = perm.shape[0]
    ret = torch.empty_like(perm)
    ret[perm] = torch.arange(N, 
        dtype=torch.long, 
        device=perm.device)
    return ret

In [65]:
dataset = PygLinkPropPredDataset(name="ogbl-collab")
data = dataset[0]
es = dataset.get_edge_split()

In [66]:
for key1 in es:
    for key2 in es[key1]:
        print(key1, key2, es[key1][key2].shape)

train edge torch.Size([1179052, 2])
train weight torch.Size([1179052])
train year torch.Size([1179052])
valid edge torch.Size([60084, 2])
valid weight torch.Size([60084])
valid year torch.Size([60084])
valid edge_neg torch.Size([100000, 2])
test edge torch.Size([46329, 2])
test weight torch.Size([46329])
test year torch.Size([46329])
test edge_neg torch.Size([100000, 2])


In [67]:
cld = ClusterData(data, 25)

Computing METIS partitioning...
Done!


In [68]:
assert (cld.data.x == data.x[cld.perm]).all().item()

In [69]:
revperm = reverseperm(cld.perm)
nei = revperm[data.edge_index]
nx = data.x[cld.perm]
nea = data.edge_weight

In [70]:
neibin = torch.searchsorted(cld.partptr, nei)#torch.stack(cld.data.adj.coo()[:-1]))
print((neibin[0]==neibin[1]).float().mean())
tnnode = torch.diff(cld.partptr)
tnedge = degree(neibin[:, neibin[0]==neibin[1]][0])

tensor(0.8928)


In [71]:
tnnode

tensor([9179, 9534, 9187, 9693, 9522, 9698, 9718, 9178, 9718, 9159, 9708, 9704,
        9161, 9694, 9697, 9161, 9676, 9159, 9159, 9194, 9703, 9700, 9170, 9225,
        9171])

In [72]:
tnedge[1:]/torch.square(tnnode)

tensor([0.0008, 0.0008, 0.0008, 0.0009, 0.0007, 0.0013, 0.0006, 0.0008, 0.0009,
        0.0021, 0.0011, 0.0019, 0.0008, 0.0009, 0.0010, 0.0007, 0.0009, 0.0008,
        0.0007, 0.0009, 0.0008, 0.0008, 0.0008, 0.0009, 0.0009])

In [73]:
nes = {"valid":{}, "test":{}}
nes["valid"]["edge"] = revperm[es["valid"]["edge"]]
nes["valid"]["edge_neg"] = revperm[es["valid"]["edge_neg"]]
nes["test"]["edge"] = revperm[es["test"]["edge"]]
nes["test"]["edge_neg"] = revperm[es["test"]["edge_neg"]]

In [74]:
bin = {"valid":{}, "test":{}}
for key1 in nes:
    for key2 in nes[key1]:
        bin[key1][key2] = torch.searchsorted(cld.partptr, nes[key1][key2])

In [75]:
def hashbin(bin, num_part):
    return bin[:, 0] * num_part + bin[:, 1]

In [76]:
for key1 in bin:
    for key2 in bin[key1]:
        tb = bin[key1][key2]
        print(hashbin(tb, cld.num_parts).unique().shape[0]/cld.num_parts**2)
        print((tb[:, 0]==tb[:, 1]).float().mean())

0.9696
tensor(0.6549)
1.0
tensor(0.0399)
0.9664
tensor(0.5843)
1.0
tensor(0.0399)


In [77]:
from ogb.linkproppred import Evaluator

In [78]:
evaluator = Evaluator(name="ogbl-collab")

In [79]:
tb1 = bin["test"]["edge"].t()
tb1 = (tb1[0]==tb1[1]).float()
tb2 = bin["test"]["edge_neg"].t()
tb2 = 0.5*(tb2[0]==tb2[1]).float()
evaluator.eval({'y_pred_pos': tb1,
            'y_pred_neg': tb2})

{'hits@50': 0.584320835761618}